In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import load_model
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import load_model 

In [3]:
model = load_model("LSTM_Mel_Spec-05.keras")

In [4]:
# model = load_model("CNN_Spec_DB-05.keras")

In [5]:
real_path = r"\for-2seconds\testing\real"
fake_path = r"\for-2seconds\testing\fake"

real = []
fake = []

mreal_path = r"\for-2seconds\mixed\real"
mfake_path = r"\for-2seconds\mixed\fake"

In [24]:
# Spec 0.5

# def get_data(folder_path):
#     specs = []    
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         audio_parts = np.array_split(audio, 4)
#         for part in audio_parts:
#             spectrogram = abs(librosa.stft(y=part))
#             spectrogram = librosa.amplitude_to_db(spectrogram, ref=np.max)
#             specs.append(spectrogram)
#     return specs

# Mel-spectrogram 0.5

def get_data(folder_path):
    mel_specs = []
    for filename in os.listdir(folder_path):
        audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
        audio_parts = np.array_split(audio, 4)
        for part in audio_parts:
            mel_spec = librosa.feature.melspectrogram(y=part, sr=sr)
            # mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
            mel_specs.append(mel_spec)
    return mel_specs


In [25]:
# real = get_data(mreal_path)
# fake = get_data(mfake_path)

In [26]:
real = get_data(real_path)
fake = get_data(fake_path)

In [27]:
testR=np.array(real)
testF=np.array(fake)
print(len(real))

predR = model.predict(testR)
predF = model.predict(testF)

2176
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [28]:
threshold = 0.5

binary_predictionsR = [1 if pr < threshold else 0 for pr in predR]

real_count = sum(binary_predictionsR)

binary_predictionsF = [1 if pr > threshold else 0 for pr in predF]

fake_count = sum(binary_predictionsF)

print("Кількість справжніх:", real_count, "з", len(predR))
print("Кількість фальшивих:", fake_count, "з", len(predF))
acc = (real_count + fake_count) / (len(real)+len(fake))
acc = "{:.2f}".format(acc*100)
print(acc + "%")

Кількість справжніх: 1584 з 2176
Кількість фальшивих: 1652 з 2176
74.36%


In [29]:
from sklearn.metrics import confusion_matrix
real_true_labels = [0] * len(real)
fake_true_labels = [1] * len(fake)

all_predicted_labels = np.concatenate((predR, predF))
all_predicted_labels = np.where(all_predicted_labels < 0.5, 0, 1)
all_true_labels = real_true_labels + fake_true_labels

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)
print(conf_matrix)

[[1584  592]
 [ 524 1652]]


In [30]:
import pandas as pd

# pd.DataFrame(predR).to_csv('LSTM_MpredR.csv', index=False)
# pd.DataFrame(predF).to_csv('LSTM_MpredF.csv', index=False)

pd.DataFrame(predR).to_csv('LSTM_predR.csv', index=False)
pd.DataFrame(predF).to_csv('LSTM_predF.csv', index=False)

# loaded_predR = pd.read_csv('predR.csv').values
# loaded_predF = pd.read_csv('predF.csv').values


In [6]:
loaded_predR = pd.read_csv('MpredR.csv').values
loaded_predF = pd.read_csv('MpredF.csv').values

LSTMloaded_predR = pd.read_csv('LSTM_MpredR.csv').values
LSTMloaded_predF = pd.read_csv('LSTM_MpredF.csv').values

# loaded_predR = pd.read_csv('LSTM_MpredR.csv').values
# loaded_predF = pd.read_csv('LSTM_MpredF.csv').values

In [7]:
loaded_predR

array([[4.1893510e-03],
       [4.1126664e-04],
       [8.3303110e-05],
       ...,
       [2.3669256e-01],
       [2.3844436e-01],
       [4.1806522e-01]])

In [8]:
loaded_predF

array([[0.8749963],
       [0.1783164],
       [0.9254462],
       ...,
       [1.       ],
       [1.       ],
       [1.       ]])

In [9]:
groupR1 = np.reshape(loaded_predR, (-1, 4))
groupR2 = np.reshape(LSTMloaded_predR, (-1, 4))


groupF1 = np.reshape(loaded_predF, (-1, 4))
groupF2 = np.reshape(LSTMloaded_predF, (-1, 4))


combined1 = np.hstack((groupedR1, groupedR2))
combined2 = np.hstack((groupedF1, groupedF2))

meansR = np.mean(combined1, axis=1)

meansF = np.mean(combined2, axis=1)


In [10]:
threshold = 0.5

binary_predictionsR = [1 if pr < threshold else 0 for pr in meansR]

real_count = sum(binary_predictionsR)

binary_predictionsF = [1 if pr > threshold else 0 for pr in meansF]

fake_count = sum(binary_predictionsF)

print("Кількість справжніх:", real_count, "з", len(meansR))
print("Кількість фальшивих:", fake_count, "з", len(meansF))
acc = (real_count + fake_count) / (len(meansR)+len(meansF))
acc = "{:.2f}".format(acc*100)
print(acc + "%")

Кількість справжніх: 1597 з 1614
Кількість фальшивих: 1562 з 1614
97.86%
